In [2]:
"""
None:Pythonic missing data
"""

import pandas as pd
import numpy as np

#None objects as missing values
arr1=np.array([1.1,2,3,4])
arr1.dtype
arr2=np.array([1,2,3,4])
arr2.dtype

dtype('int32')

In [3]:
#python
arr1.sum()

10.1

In [4]:
"""
Nan Missing Numerical data
"""

arr3=np.array([1,np.nan,3,4])
type(np.nan)

float

In [5]:
#Arithmetic wuth Nan will be anoter nan
arr3.sum()

nan

In [6]:
#special numpy aggeregation func that ignore these missing vaule
np.nanmin(arr3)

1.0

In [7]:
#pandas automatically converts the non to a nan value
ser=pd.Series([1,np.nan,'2',None])
ser

0       1
1     NaN
2       2
3    None
dtype: object

In [8]:
"""
Detecting numm value
"""
ser=pd.Series([1,np.nan,'hello',None])
print(ser,'\n')
ser.isnull #notnull하면 값이 거꾸로 바뀜

0        1
1      NaN
2    hello
3     None
dtype: object 



<bound method Series.isnull of 0        1
1      NaN
2    hello
3     None
dtype: object>

In [9]:
"""
Dropping null values
"""
# dropna()
ser.dropna()

0        1
2    hello
dtype: object

In [11]:
# For a DataFrame, there are more options

df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [13]:
#df.dropna(): list-wise deletion
df.dropna()

,0,1,2
1,2.0,3.0,5


In [14]:
# df.dropna(axis='columns'): variable deletion
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [15]:
# how/thresh parameters
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [16]:
# how='any' (default)
# how='all' which will only drop rows/columns that are all null values
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [17]:
# thresh = minimum number of non-null values to be kept
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [18]:
"""
Filling Null Values
"""

ser = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
ser

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [19]:
# Fill null values with a certain value
ser.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [20]:
# Forward-fill = LOCF
ser.fillna(method='ffill') # equals to "ser.ffill()"

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [21]:
# backward-fill = NOCB
ser.fillna(method='bfill') # equals to "ser.bfill()"

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [24]:
# bfill with rows
df.fillna(method='bfill', axis='rows')

,0,1,2,3
0,1.0,3.0,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [53]:
from pandas import datetime
from matplotlib import pyplot as plt

<ipython-input-53-183261162689>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [38]:
"""
Load AirQualityUCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

FileNotFoundError: [Errno 2] No such file or directory: './data/AirQualityUCI_refined.csv'

In [30]:
# Print the summary of the dataset

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2 non-null      float64
 1   1       2 non-null      float64
 2   2       3 non-null      int64  
 3   3       0 non-null      float64
dtypes: float64(3), int64(1)
memory usage: 224.0 bytes


In [36]:
# Visualization setup
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # set plot styles
%config InlineBackend.figure_format = 'svg'

In [32]:
# Visualize the series of CO(GT)

df['CO(GT)'].plot()

KeyError: 'CO(GT)'

In [33]:
# imputation

imp_locf = df['CO(GT)'].copy().ffill()
imp_nocb = df['CO(GT)'].copy().bfill()
imp_linear = df['CO(GT)'].copy().interpolate()
imp_mean = df['CO(GT)'].copy().fillna(df['CO(GT)'].mean())

KeyError: 'CO(GT)'

In [34]:
# k-nn imputation

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)    # default: 2
imp_knn = df.copy().values
imp_knn = imputer.fit_transform(imp_knn)

In [35]:
# add indices to the imputed result of k-nn

imp_df = pd.DataFrame(imp_knn, index=imp_locf.index, columns=df.columns)

NameError: name 'imp_locf' is not defined

In [39]:
# Visualizing the imputed results

plt.plot(df['CO(GT)'], label='actual', zorder=10)
plt.plot(imp_locf, label='locf', zorder=1)
plt.plot(imp_nocb, label='nocb', zorder=2)
plt.plot(imp_linear, label='linear interpolation', zorder=3)
plt.plot(imp_mean, label='mean substitution', zorder=4)
plt.plot(imp_df['CO(GT)'], label='k-nearest neighbor', zorder=5)
plt.legend(loc='best')
plt.show()

KeyError: 'CO(GT)'

In [40]:
# Select the certain period to visualize

start = '2004-07-18'
end = '2004-10-20'

In [41]:
# Visualize 2004-07 ~ 2004-10

plt.plot(df['CO(GT)'].loc[start:end], label='actual', zorder=10)
plt.plot(imp_locf.loc[start:end], label='locf', zorder=1)
plt.plot(imp_nocb.loc[start:end], label='nocb', zorder=2)
plt.plot(imp_linear.loc[start:end], label='linear interpolation', zorder=3)
plt.plot(imp_mean.loc[start:end], label='mean substitution', zorder=4)
plt.plot(imp_df['CO(GT)'].loc[start:end], label='k-nearest neighbor', zorder=5)
plt.legend(loc='best')
plt.show()

KeyError: 'CO(GT)'